In [1]:
import os
import numpy as np
import json
import pandas as pd
import torch
import pickle

from datasets import load_from_disk
from transformers import AutoTokenizer
from input.code.dpr.trainer_DPR import BiEncoderTrainer # trainer_DPR 모듈 위치에 따라서, from을 수정해주세요
from input.code.dpr.cls_Encoder import BertEncoder, RoBertaEncoder # cls_Encoder 모듈 위치에 따라서, from을 수정해주세요

In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "input", "data")
P_ENCODER_DIR = os.path.join(BASE_DIR, "input", "code", "dpr", "roberta","p_encoder")
Q_ENCODER_DIR = os.path.join(BASE_DIR, "input", "code", "dpr", "roberta", "q_encoder")
datasets = load_from_disk(os.path.join(DATA_DIR, "train_dataset"))

In [ ]:
with open(os.path.join(DATA_DIR, "wikipedia_documents.json"), "r") as f:
	wiki_corpus = json.load(f)
wiki_corpus = list(dict.fromkeys([v['text'] for v in wiki_corpus.values()]))

In [3]:
model_name = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
p_encoder = BertEncoder.from_pretrained(model_name).to("cuda:0")
q_encoder = BertEncoder.from_pretrained(model_name).to("cuda:0")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

In [5]:
bitrainer = BiEncoderTrainer(p_encoder=p_encoder,
							 q_encoder=q_encoder,
							 tokenizer=tokenizer,
							 epochs=3,
							 batch_size=30,
							 neg_num=2,
							 lr=5e-5,
							 train_datasets=datasets['train'],
							 eval_datasets=datasets['validation'],
							 contexts_document=wiki_corpus,)

In [6]:
bitrainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch: 100%|██████████| 3/3 [04:12<00:00, 84.27s/it]


In [7]:
sample_idx = np.random.choice(range(len(datasets['validation'])), 5)

In [8]:
passage_dataset = datasets['validation'][sample_idx]
query = datasets['validation'][sample_idx]['question']

In [9]:
score, pred_rank, pred_corpus = bitrainer.predict(passage_dataset, query)

/opt/ml/input/code/dpr/trainer_DPR.py:194: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  p_embs = torch.Tensor(p_embs).squeeze()


In [10]:
for i in range(5):
	print("[Query] : ", query[i])
	print("[True Passage] \n", passage_dataset['context'][i])
	for k in range(5):
		print(f"Top-{k+1} Passage")
		print("[Score] : ", score[i][k])
		print("[Predicted Passage] \n", pred_corpus[i][k])
		print("=" * 15)

[Query] :  인형사'를 만들어낸 것으로 추측되는 사업의 이름은?
[True Passage] 
 2029년의 지구는 비약적으로 발달한 기술 문명을 바탕으로 전 세계가 전자 통신망으로 연결되어 있는 곳이다. 공각기동대는 2029년을 전후하여 활동하는 정부의 특수 기관으로, 작품 내에서는 지금도 일본에 실재하는 기구인 국가공안위원회 소속 가상 집단 '공안 9과'의 다른 이름이다. 이 집단은 전자 네트워크 해킹이나 생화학 무기 등등 최첨단 기술에 근거하였거나 연관되어 있는 범죄에 대응하는 일에 특화되어 있으며, 각각 야전 군인이나 특수 기동대원, 일본 경찰청 소속 형사 등등 다양한 경력을 가진 인물들로 구성되어 있다. 흔히 소령이라는 별명으로 불리며 대원들을 실전 지휘하는 여성 쿠사나기 모토코가 주인공으로 등장한다. 모토코의 몸은 뇌 일부를 제외하면 모두 인공적인 기계 장치로 되어 있으며 다른 어떠한 사람보다도 이식한 기계 몸을 제어하는 능력이 탁월한데, 그녀의 몸이 자의로 개조된 것인지는 불분명하다.\n\n오시 마모루가 감독한 영화와는 달리 마사무네의 원작 만화는 우스개나 단편적인 에피소드가 섞여 있는 비교적 가벼운 작품이다. 물론 이 작품에서도 철학적이고 심오한 주제의 사색은 등장하는데, 중요한 갈등은 두 가지로 나누어 볼 수 있다. 하나는 본디 신체를 거의 잃고 몸의 대부분이 기계로 되어 있는 모토코의 내적 갈등이다. 그녀의 고민은 테세우스의 배와 비슷한 문제로 볼 수 있는데, 대부분의 사람이 자기 자신을 규정지을 때 쉽게 사용할 수 있는 근거가 되는 신체가 그녀의 경우 몇 번이나 개조되고 일부가 교체되었기 때문에 그녀에게는 이렇다 할 의미가 없으며, 따라서 모토코는 '나'란 누구인지, 쿠사나기 모토코의 실체는 무형의 정신인지 아니면 전자 기판과 마이크로프로세서, 몇 그램의 뇌세포로 이루어진 사고 회로인지 고민한다. 다른 한 가지 갈등은 인공 지능을 만들어낼 목적인 것으로 알려진 '일급 비밀 프로젝트 2501'의 결과물로 나타난 것으로 보이는 수준급의 해